# ETH-BTC Cointegration

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz


In [2]:
# Adjusting the start and end times
frequency = "1h"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=90)).strftime('%Y-%m-%d')  # 5 days ago from the current date
end_time = datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d')  # Current date in UTC

In [3]:
# Define your tickers, start time, end time, and frequency
tickers = ["BTC-USD", "ETH-USD"]

# Retrieve historical data and adjust to Pacific Time
historical_data = {}
for ticker in tickers:
    data = yf.download(ticker, start=start_time, end=end_time, interval=frequency)[['High', 'Low', 'Open', 'Close']]
    #data.index = data.index.tz_localize('UTC').tz_convert('US/Pacific')  # Convert timezone to Pacific Time
    historical_data[ticker] = data

# Combine data without weights
combined_data = pd.DataFrame()
for ticker, data in historical_data.items():
    for col in data.columns:
        combined_column_name = f"{ticker}_{col}"  # Create a unique column name for each 'High' and 'Low'
        combined_data[combined_column_name] = data[col]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
combined_data.dropna(inplace=True)
combined_data

,BTC-USD_High,BTC-USD_Low,BTC-USD_Open,BTC-USD_Close,ETH-USD_High,ETH-USD_Low,ETH-USD_Open,ETH-USD_Close
Datetime,,,,,,,,
2023-11-07 00:00:00+00:00,35064.667969,34870.718750,35047.792969,34955.625000,1904.331055,1887.138672,1900.599731,1893.568848
2023-11-07 01:00:00+00:00,34971.332031,34862.753906,34947.074219,34954.566406,1897.382690,1890.416016,1893.077393,1893.822021
2023-11-07 02:00:00+00:00,34983.476562,34832.468750,34951.878906,34832.468750,1896.502808,1881.861938,1893.922974,1881.861938
2023-11-07 03:00:00+00:00,34929.789062,34822.203125,34830.269531,34929.789062,1890.729370,1879.480103,1881.353882,1890.729370
2023-11-07 04:00:00+00:00,34950.535156,34842.265625,34930.542969,34918.316406,1895.040527,1886.750610,1890.775391,1894.012817
...,...,...,...,...,...,...,...,...
2024-02-04 19:00:00+00:00,42804.558594,42593.351562,42786.347656,42636.054688,2294.580811,2272.304932,2294.041748,2279.523682
2024-02-04 20:00:00+00:00,42892.562500,42578.277344,42629.753906,42840.046875,2307.472656,2275.073975,2279.201660,2302.521240
2024-02-04 21:00:00+00:00,42932.992188,42758.410156,42860.742188,42758.410156,2308.040039,2300.000000,2302.647949,2300.177490


In [5]:
#combined_data = combined_data.pct_change() * 100
combined_data.dropna(inplace=True)

In [6]:
import pandas as pd
from statsmodels.tsa.stattools import coint

# Assuming combined_data is your DataFrame with the following columns:
# ['BTC-USD_Open', 'BTC-USD_High', 'BTC-USD_Low', 'BTC-USD_Close', 
# 'ETH-USD_Open', 'ETH-USD_High', 'ETH-USD_Low', 'ETH-USD_Close']

# Test for cointegration between BTC and ETH Open prices
score_open, p_value_open, crit_value_open = coint(combined_data['BTC-USD_Open'], combined_data['ETH-USD_Open'])

# Test for cointegration between BTC and ETH High prices
score_high, p_value_high, crit_value_high = coint(combined_data['BTC-USD_High'], combined_data['ETH-USD_High'])

# Test for cointegration between BTC and ETH Low prices
score_low, p_value_low, crit_value_low = coint(combined_data['BTC-USD_Low'], combined_data['ETH-USD_Low'])

# Test for cointegration between BTC and ETH Close prices
score_close, p_value_close, crit_value_close = coint(combined_data['BTC-USD_Close'], combined_data['ETH-USD_Close'])

# Print the results for Open prices
print("Cointegration Test between BTC and ETH Open Prices:")
print(f"Test Statistic: {score_open}")
print(f"P-value: {p_value_open}")
print(f"Critical Values: {crit_value_open}")
if p_value_open < 0.05:
    print("Result: The Open prices of BTC and ETH are cointegrated at the 5% significance level.\n")
else:
    print("Result: The Open prices of BTC and ETH are not cointegrated at the 5% significance level.\n")

# Print the results for High prices
print("Cointegration Test between BTC and ETH High Prices:")
print(f"Test Statistic: {score_high}")
print(f"P-value: {p_value_high}")
print(f"Critical Values: {crit_value_high}")
if p_value_high < 0.05:
    print("Result: The High prices of BTC and ETH are cointegrated at the 5% significance level.\n")
else:
    print("Result: The High prices of BTC and ETH are not cointegrated at the 5% significance level.\n")

# Print the results for Low prices
print("Cointegration Test between BTC and ETH Low Prices:")
print(f"Test Statistic: {score_low}")
print(f"P-value: {p_value_low}")
print(f"Critical Values: {crit_value_low}")
if p_value_low < 0.05:
    print("Result: The Low prices of BTC and ETH are cointegrated at the 5% significance level.\n")
else:
    print("Result: The Low prices of BTC and ETH are not cointegrated at the 5% significance level.\n")

# Print the results for Close prices
print("Cointegration Test between BTC and ETH Close Prices:")
print(f"Test Statistic: {score_close}")
print(f"P-value: {p_value_close}")
print(f"Critical Values: {crit_value_close}")
if p_value_close < 0.05:
    print("Result: The Close prices of BTC and ETH are cointegrated at the 5% significance level.")
else:
    print("Result: The Close prices of BTC and ETH are not cointegrated at the 5% significance level.")


Cointegration Test between BTC and ETH Open Prices:
Test Statistic: -2.414866127681278
P-value: 0.3178467273286948
Critical Values: [-3.90152222 -3.33896284 -3.04641592]
Result: The Open prices of BTC and ETH are not cointegrated at the 5% significance level.

Cointegration Test between BTC and ETH High Prices:
Test Statistic: -2.1427220102412696
P-value: 0.45406664358200655
Critical Values: [-3.90152222 -3.33896284 -3.04641592]
Result: The High prices of BTC and ETH are not cointegrated at the 5% significance level.

Cointegration Test between BTC and ETH Low Prices:
Test Statistic: -2.1993155871516756
P-value: 0.4247164776834072
Critical Values: [-3.90152222 -3.33896284 -3.04641592]
Result: The Low prices of BTC and ETH are not cointegrated at the 5% significance level.

Cointegration Test between BTC and ETH Close Prices:
Test Statistic: -2.412654627265541
P-value: 0.3188850964843305
Critical Values: [-3.90152222 -3.33896284 -3.04641592]
Result: The Close prices of BTC and ETH are n